In [1]:
from shapely.geometry import Point
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import folium
from folium.plugins import MarkerCluster
from folium.plugins import FastMarkerCluster
import requests

1. A dataset containing details about Metro Nashville Police Department reported incidents is available at https://data.nashville.gov/Police/Metro-Nashville-Police-Department-Incidents/2u6v-ujjs. 

Make use of the API to find all aggravated burglary incidents that were reported during the six month period from January 1, 2021 through June 30, 2021.

In [2]:
endpoint = 'https://data.nashville.gov/resource/2u6v-ujjs.json'

In [3]:
params = {
    'offense_description': 'BURGLARY- AGGRAVATED',
    '$where': 'incident_reported between \'2021-01-01\' AND \'2021-06-30\'',
    '$limit': '50000'
}

In [4]:
response = requests.get(endpoint,
                        params = params )

In [5]:
response

<Response [200]>

In [70]:
response.json()

[{'primary_key': '20210249540_11',
  'incident_number': '20210249540',
  'report_type': 'D',
  'report_type_description': 'DISPATCHED',
  'incident_status_code': 'O',
  'incident_status_description': 'OPEN',
  'investigation_status': 'Open',
  'incident_occurred': '2021-05-04T23:45:00.000',
  'incident_reported': '2021-05-05T00:45:00.000',
  'incident_location': 'UNIVERSITY CT',
  'latitude': '36.15',
  'longitude': '-86.77',
  'rpa': '8203',
  'zone': '511',
  'location_code': '22',
  'location_description': 'RESIDENCE, HOME',
  'offense_number': '1',
  'offense_nibrs': '220',
  'offense_description': 'BURGLARY- AGGRAVATED',
  'weapon_primary': '09',
  'weapon_description': 'PERSONAL (HANDS)',
  'victim_number': '1',
  'domestic_related': False,
  'victim_type': 'I',
  'victim_description': 'INDIVIDUAL (18 AND OVER)',
  'victim_gender': 'F',
  'victim_race': 'B',
  'victim_ethnicity': 'Non-Hispanic',
  'victim_county_resident': 'RESIDENT',
  'mapped_location': {'type': 'Point', 'coord

In [7]:
raw = response.json()

In [8]:
burglaries = pd.DataFrame.from_dict(raw)

burglaries.head()

,primary_key,incident_number,report_type,report_type_description,incident_status_code,incident_status_description,investigation_status,incident_occurred,incident_reported,incident_location,...,victim_number,domestic_related,victim_type,victim_description,victim_gender,victim_race,victim_ethnicity,victim_county_resident,mapped_location,zip_code
0,20210249540_11,20210249540,D,DISPATCHED,O,OPEN,Open,2021-05-04T23:45:00.000,2021-05-05T00:45:00.000,UNIVERSITY CT,...,1,False,I,INDIVIDUAL (18 AND OVER),F,B,Non-Hispanic,RESIDENT,"{'type': 'Point', 'coordinates': [-86.77, 36.15]}",NaN
1,20210248840_11,20210248840,D,DISPATCHED,O,OPEN,Open,2021-05-04T12:06:00.000,2021-05-04T14:33:00.000,BONNAFAIR DR,...,1,False,I,INDIVIDUAL (18 AND OVER),F,W,Non-Hispanic,RESIDENT,"{'type': 'Point', 'coordinates': [-86.63, 36.2]}",NaN
2,20210244919_11,20210244919,D,DISPATCHED,O,OPEN,Open,2021-05-01T23:00:00.000,2021-05-02T12:40:00.000,8TH AVE S,...,1,False,I,INDIVIDUAL (18 AND OVER),M,W,Non-Hispanic,NON RESIDENT,"{'type': 'Point', 'coordinates': [-86.78, 36.14]}",NaN
3,20210244892_13,20210244892,D,DISPATCHED,O,OPEN,Open,2021-05-01T21:30:00.000,2021-05-02T11:57:00.000,EAST ARGYLE AVE,...,3,False,I,INDIVIDUAL (18 AND OVER),M,W,Non-Hispanic,NON RESIDENT,"{'type': 'Point', 'coordinates': [-86.78, 36.14]}",NaN
4,20210249540_12,20210249540,D,DISPATCHED,O,OPEN,Open,2021-05-04T23:45:00.000,2021-05-05T00:45:00.000,UNIVERSITY CT,...,2,False,I,INDIVIDUAL (18 AND OVER),M,B,Non-Hispanic,RESIDENT,"{'type': 'Point', 'coordinates': [-86.77, 36.15]}",NaN


In [9]:
burglaries.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1046 entries, 0 to 1045
Data columns (total 31 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   primary_key                  1046 non-null   object
 1   incident_number              1046 non-null   object
 2   report_type                  1046 non-null   object
 3   report_type_description      1034 non-null   object
 4   incident_status_code         1046 non-null   object
 5   incident_status_description  1046 non-null   object
 6   investigation_status         1046 non-null   object
 7   incident_occurred            1046 non-null   object
 8   incident_reported            1046 non-null   object
 9   incident_location            1044 non-null   object
 10  latitude                     1044 non-null   object
 11  longitude                    1044 non-null   object
 12  rpa                          964 non-null    object
 13  zone                         964 

In [10]:
burglaries['incident_number'].value_counts()

20210323659    9
20210309042    8
20210131522    7
20210240530    7
20210244892    6
              ..
20210063395    1
20210031613    1
20210120747    1
20210074193    1
20210257587    1
Name: incident_number, Length: 824, dtype: int64

In [11]:
burglaries_single = burglaries[burglaries['victim_number'] == '1']

In [83]:
conv = {
    'incident_occurred': 'datetime64',
    'incident_reported': 'datetime64',
    'latitude': 'float64',
    'longitude': 'float64'
}

burglaries_single = burglaries_single.astype(conv)

In [85]:
burglaries_single['geometry'] = burglaries_single.apply(lambda x: Point((x.longitude, 
                                                                         x.latitude)), 
                                                        axis=1)

In [86]:
burglaries_single.head()

,primary_key,incident_number,report_type,report_type_description,incident_status_code,incident_status_description,investigation_status,incident_occurred,incident_reported,incident_location,...,domestic_related,victim_type,victim_description,victim_gender,victim_race,victim_ethnicity,victim_county_resident,mapped_location,zip_code,geometry
0,20210249540_11,20210249540,D,DISPATCHED,O,OPEN,Open,2021-05-04 23:45:00,2021-05-05 00:45:00,UNIVERSITY CT,...,False,I,INDIVIDUAL (18 AND OVER),F,B,Non-Hispanic,RESIDENT,"{'type': 'Point', 'coordinates': [-86.77, 36.15]}",NaN,POINT (-86.77 36.15)
1,20210248840_11,20210248840,D,DISPATCHED,O,OPEN,Open,2021-05-04 12:06:00,2021-05-04 14:33:00,BONNAFAIR DR,...,False,I,INDIVIDUAL (18 AND OVER),F,W,Non-Hispanic,RESIDENT,"{'type': 'Point', 'coordinates': [-86.63, 36.2]}",NaN,POINT (-86.63 36.2)
2,20210244919_11,20210244919,D,DISPATCHED,O,OPEN,Open,2021-05-01 23:00:00,2021-05-02 12:40:00,8TH AVE S,...,False,I,INDIVIDUAL (18 AND OVER),M,W,Non-Hispanic,NON RESIDENT,"{'type': 'Point', 'coordinates': [-86.78, 36.14]}",NaN,POINT (-86.78 36.14)
6,20210248402_11,20210248402,D,DISPATCHED,O,OPEN,Open,2021-05-04 00:00:00,2021-05-04 09:50:00,NEESE DR,...,False,I,INDIVIDUAL (18 AND OVER),F,W,Non-Hispanic,RESIDENT,"{'type': 'Point', 'coordinates': [-86.73, 36.11]}",NaN,POINT (-86.73 36.11)
7,20210241953_11,20210241953,D,DISPATCHED,O,OPEN,Open,2021-04-08 14:00:00,2021-04-30 17:52:00,21ST AVE N,...,False,I,INDIVIDUAL (18 AND OVER),M,W,Non-Hispanic,RESIDENT,"{'type': 'Point', 'coordinates': [-86.81, 36.16]}",NaN,POINT (-86.81 36.16)


In [117]:
burglaries_geo = gpd.GeoDataFrame(burglaries_single, 
                           geometry = burglaries_single['geometry'])

In [118]:
burglaries_geo = burglaries_geo.set_crs(4326, allow_override=True)

In [119]:
burglaries_geo.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [120]:
geo burglaries_geo['geometry'].dropna()

0       POINT (-86.77000 36.15000)
1       POINT (-86.63000 36.20000)
2       POINT (-86.78000 36.14000)
6       POINT (-86.73000 36.11000)
7       POINT (-86.81000 36.16000)
                   ...            
1041    POINT (-86.64700 36.05200)
1042    POINT (-86.76500 36.15200)
1043    POINT (-86.80700 36.20400)
1044    POINT (-86.72000 36.05000)
1045    POINT (-86.71000 36.27000)
Name: geometry, Length: 811, dtype: geometry

2. Download the 2019 census tract shapefiles for Tennessee from https://www.census.gov/geographies/mapping-files/time-series/geo/tiger-line-file.2019.html. (The FIPS code for Tennessee is 47). 

Perform a spatial join to determine the census tract in which each burglary incident occurred. Which census tract had the highest number of burglaries? Warning - each incident can appear multiple times if there are multiple victims, so be sure that you aren't double-counting any incidents.

In [104]:
tracts = gpd.read_file('../data/tl_2019_47_tract.shp')

tracts.head()

,STATEFP,COUNTYFP,TRACTCE,GEOID,NAME,NAMELSAD,MTFCC,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,geometry
0,47,165,021002,47165021002,210.02,Census Tract 210.02,G5020,S,30654374,207994,+36.3521602,-086.6464309,"POLYGON ((-86.69927 36.32775, -86.69923 36.327..."
1,47,165,020901,47165020901,209.01,Census Tract 209.01,G5020,S,23811766,3412697,+36.3483143,-086.4070907,"POLYGON ((-86.43949 36.35962, -86.43915 36.359..."
2,47,165,020300,47165020300,203,Census Tract 203,G5020,S,14629292,0,+36.5796910,-086.5114955,"POLYGON ((-86.53716 36.56852, -86.53713 36.568..."
3,47,165,020902,47165020902,209.02,Census Tract 209.02,G5020,S,9756715,0,+36.3724350,-086.4411274,"POLYGON ((-86.47106 36.38238, -86.47081 36.382..."
4,47,165,020800,47165020800,208,Census Tract 208,G5020,S,10235406,37743,+36.4005117,-086.4528100,"POLYGON ((-86.48786 36.37320, -86.48783 36.373..."


In [107]:
tracts = tracts.set_crs(burglaries_geo.crs, allow_override = True)

tracts.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [116]:
burglaries_by_tract = gpd.sjoin(tracts, burglaries_geo, op = 'within')

burglaries_by_tract.head()

ValueError: 'right_df' should be GeoDataFrame, got <class 'geopandas.geoseries.GeoSeries'>